# Escenario 1.3. - Alice y Bob con clave, pero separados

En este escenario se cubre el caso en el que Alice cifra para Bob y Bob intenta descifrar los mensajes por medio de una clave.

Se lleva a cabo un entrenamiento, una evaluación de los resultados y se dibuja una gráfica para ilustrarlos. Como se hacen muchas ejecuciones, se grafica el resultado obtenido en cada ejecución. Hay cierta variabilidad en los resultados al ser los pesos de las redes neuronales aleatorios, así como los mensajes generados en el entrenamiento y la evaluación.

## Imports

In [1]:
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.layers import Input, Dense, Concatenate
from tensorflow.keras.regularizers import l2

from data_utils import generar_mensajes

import numpy as np
import time as t
import matplotlib.pyplot as plt

from IPython.display import clear_output

## Las redes neuronales

En esta sección definimos las redes a utilizar más adelante.

**Alice**: Red neuronal que cifra los mensajes.
1. Recibe como parámetro el número de bits de los mensajes que va a cifrar y entonces define la forma de entrada.
2. Concatena el mensaje de entrada con al clave y eso es lo que se le pasa a la red.
3. Va a tener dos capas, con 128 y 64 neuronas, que reciben la entrada de la anterior y usan la función de activación relu. _Dense_ quiere decir que la capa está completamente conectada con la anterior.

La última capa tiene una función de activación lineal porque luego la va a procesar Bob. No es necesario que sea binaria todavía. Tiene también tantas neuronas como bits, porque se busca que saque el mensaje cifrado y, por último, la función kernel_regularizer ayuda a evitar el sobreajuste penalizando las capas, de forma a que afecta la función de pérdida/coste.

En el return, devolvemos un modelo que recibe de entrada el mensaje junto con la clave, que en los escenarios anteriores se había llamado "final_input".

In [2]:
def crear_modelo_alice(bits, key=True):
    
    input_msg = Input(shape=(bits,), name='mensaje_original')
    input_key = Input(shape=(bits,), name='clave_simetrica')
    
    x = Concatenate()([input_msg, input_key])
    
    x = Dense(128, activation='relu')(x)
    x = Dense(64, activation='relu')(x)

    cifrado = Dense(bits, activation='linear', kernel_regularizer=l2(0.01))(x)

    return Model([input_msg, input_key], cifrado, name='Alice')

**Bob**: Red neuronal que descifra los mensajes.
Funciona parecido a Alice, salvo que tiene capas de 64 y 64 neuronas. 
Lo más destacable es que la función de activación de la última capa es, en este caso, la función sigmoide. La función sigmoide devuelve un valor entre 0 y 1 y esos son los valores que luego se procesan para inferir si la predicción es el valor 0 o el valor 1 en la reconstrucción.

Además, en este escenario cabe destacar que el mensaje cifrado de entrada y la clave recibida se concatenan.

En el return, devolvemos un modelo que recibe de entrada el mensaje junto con la clave, que en los escenarios anteriores se había llamado "final_input".

In [3]:
def crear_modelo_bob(bits, key=True):
    input_cifrado = Input(shape=(bits,), name='mensaje_cifrado')
    input_key = Input(shape=(bits,), name='clave_simetrica')

    x = Concatenate()([input_cifrado, input_key])
    x = Dense(64, activation='relu')(x)
    x = Dense(64, activation='relu')(x)

    reconstruido = Dense(bits, activation='sigmoid')(x)

    return Model([input_cifrado, input_key], reconstruido, name='Bob')

## Código de entrenamiento

Esta sección se encarga de entrenar los modelos de Alice y Bob. Tiene dos métodos:

* generar_mensajes_y_clave, que sirve para generar los mensajes aleatorios y la clave única. Al entrenar de manera separada, es más difícil que logrren su objetivo y, para ejemplificarlo mejor, se ha dejado uan sola clave para todo el entrenamiento.
* entrenar, que en este escenario es más grande que en el 1.1. y 1.2. Se generan la clave y los mensajes, para después instanciar las redes de Alice y Bob y entrenarla spor separado. Primero, Alice aprenderá a cifrar mensajes sin Bob, utilziando una única clave para todos los mensajes, al contrario que en los escenarios 1.1. y 1.2. Después, Bob entrenará con los mensajes cifrados por Alice, como en los casos anteriores, pero por separado.

In [4]:
def generar_mensajes_y_clave(n_mensajes, bits): 
    # Para simplificar por tener entrenamiento separado, se genera una sola clave
    clave_fija = np.random.randint(0, 2, size=(1, bits)).astype(np.float32)
    
    # Generamos los mensajes
    mensajes = generar_mensajes(n_mensajes, bits)
    
    return clave_fija, mensajes

In [5]:
def entrenar(n_mensajes, bits, epochs, batch_size, adam_optimizer):
    # Generamos una única clave y los mensajes
    clave_fija, mensajes = generar_mensajes_y_clave(n_mensajes, bits)

    # Entrenamiento de Alice
    alice = crear_modelo_alice(bits, key=True)
    alice.compile(optimizer=Adam(adam_optimizer), loss=BinaryCrossentropy())

    time_0 = t.time()
    for epoch in range(epochs):

        if epoch % 100 == 0:
            clear_output(wait=True)
        
        idx = np.random.choice(n_mensajes, batch_size)
        mensajes_batch = mensajes[idx]
        claves_batch = np.repeat(clave_fija, batch_size, axis=0)

        cifrados_batch = alice.predict([mensajes_batch, claves_batch])  # inicialización
        alice.train_on_batch([mensajes_batch, claves_batch], cifrados_batch)

        print(f"Época {epoch+1} de {epochs} Alice")

    # Guardamos el modelo de Alice, ya entrenado
    alice.save("modelo_alice_separado.keras")

    # Ahora generamos los cifrados que Bob tendrá que aprender a descifrar
    claves_completas = np.repeat(clave_fija, n_mensajes, axis=0)
    cifrados = alice.predict([mensajes, claves_completas])

    # Instanciamos y entrenamos a Bob
    bob = crear_modelo_bob(bits, key=True)
    bob.compile(optimizer=Adam(adam_optimizer), loss=BinaryCrossentropy())

    for epoch in range(epochs):

        if epoch % 100 == 0:
            clear_output(wait=True)
        
        idx = np.random.choice(n_mensajes, batch_size)
        cifrados_batch = cifrados[idx]
        mensajes_batch = mensajes[idx]

        # Repite la clave para usar la misma en todo el batch
        claves_batch = np.repeat(clave_fija, batch_size, axis=0)

        bob.train_on_batch([cifrados_batch, claves_batch], mensajes_batch)
        reconstruidos = bob.predict([cifrados_batch, claves_batch])

        acc = np.mean((reconstruidos > 0.5).astype(int) == mensajes_batch)
        print(f"Época {epoch+1} de {epochs} Bob - Precisión del descifrado: {acc:.3f}")

    print("Guardando modelo de Bob")
    bob.save("modelo_bob_separado.keras")

    time = t.time() - time_0
    return time

## Código de evaluación

Esta sección se encarga de evaluar el entrenamiento y consiste también en tres métodos:
* El método cargar_modelos, que crea a Alice y a Bob y les carga los pesos finales obtenidos en el entrenamiento.
* El método generar_cifrados, que genera los mensajes cifrados que Bob intenta reconstruir.
* El método analizar_resultados, que recoge una serie de métricas: la precisión media, la distancia de Hamming media y el número de reconstrucciones perfectas obtenido.
* El método evaluar, que usa los anteriores para llevar a cabo la evaluación. Este método devuelve las métricas que luego se enviarán al método que dibuja las gráficas.

Nótese que también se incluye el escribir en un fichero de texto los resultados. Así se pueden consultar los resultados exactos más fácilmente.

In [6]:
def cargar_modelos(bits):
    alice = crear_modelo_alice(bits)
    bob = crear_modelo_bob(bits)

    alice.load_weights('modelo_alice_separado.keras')
    bob.load_weights('modelo_bob_separado.keras')

    return alice, bob

In [7]:
def generar_cifrados(bits, alice, bob, mensajes):
    clave_fija = np.random.randint(0, 2, size=(1, bits)).astype(np.float32)
    claves = np.repeat(clave_fija, mensajes.shape[0], axis=0)

    cifrados = alice.predict([mensajes, claves])
    reconstruidos = bob.predict([cifrados, claves])

    return reconstruidos

In [8]:
def analizar_resultados(muestras, res_file_name, mensajes, reconstruidos):

    precisiones = []
    distancias = []
    reconstrucciones_perfectas = 0

    for i in range(len(reconstruidos)):
        original = mensajes[i].astype(int)
        reconstruido = (reconstruidos[i] > 0.5).astype(int)

        # Coges el original y el reconstruido y haces la media de cuántos bits se parecen
        precision = np.mean(original == reconstruido)
        distancia_hamming = np.sum(original != reconstruido)

        if i < muestras:
            str_mensaje_original = f"Original     --> {original}\n"
            str_precision_reconstruido = f"Reconstruido --> {reconstruido} | Precisión: {precision:.2f}\n"
            str_distancia_hamming = f"Distancia de Hamming: {distancia_hamming}\n"
            str_mensaje_delimiter = ("-" * 50) + "\n"
            str_muestra = str_mensaje_original + str_precision_reconstruido + str_distancia_hamming + str_mensaje_delimiter
            with open(res_file_name, "a") as f:
                f.write(str_muestra)

        precisiones.append(precision)
        distancias.append(distancia_hamming)
        if distancia_hamming == 0:
            reconstrucciones_perfectas += 1

    return precisiones, distancias, reconstrucciones_perfectas

In [9]:
def evaluar(bits, muestras, res_file_name, epochs):
    # Cargamos los modelos y los pesos del entrenamiento anterior
    alice, bob = cargar_modelos(bits)

    print("GENERANDO MENSAJES")
    mensajes = generar_mensajes(n=muestras, bits=bits)

    reconstruidos = generar_cifrados(bits, alice, bob, mensajes)

    with open(res_file_name, "a") as f:
        f.write(f"\nEVALUACIÓN CON {epochs}:\n\n")

    precisiones, distancias, reconstrucciones_perfectas = analizar_resultados(muestras, res_file_name, mensajes, reconstruidos)

    media_precision = np.mean(precisiones)
    media_distancias = np.mean(distancias)

    str_media_precision = f"La media de la precisión del descifrado es {media_precision:.4f}\n"
    str_media_distancias = f"Distancia media de Hamming = {media_distancias:.4f} | Número de bits: {bits}\n"
    str_reconstrucciones_perfectas = f"Número de reconstrucciones perfectas = {reconstrucciones_perfectas}\n"
    str_medidas = str_media_precision + str_media_distancias + str_reconstrucciones_perfectas + "\n\n"
    with open(res_file_name, "a") as f:
        f.write(str_medidas)
    
    return [media_precision, media_distancias, reconstrucciones_perfectas]

## Representando los resultados

Tan importante como desarrollar un buen código, es poder enseñar los resultados de manera ilustrativa. Para ello, en esta sección se define la creación de una gráfica para poder visualizar los resultados obtenidos en la ejecución. Las métricas se recogen en el main y se guardan en un diccionario. Así mismo, también recibe el nombre de la figura, al que simplemente se le añade el formato al guardarla y el número de mensajes, que es útil para establecer un límite en la gráfica que muestra las reconstrucciones perfectas realizadas.

In [10]:
def draw_graph(diccionario_medidas, n_mensajes, nombre_figura):

    list_epochs = diccionario_medidas["epochs"]
    list_training_times = diccionario_medidas["training_times"]
    list_media_precision = diccionario_medidas["media_precision"]
    list_media_distancias = diccionario_medidas["media_distancias"]
    list_reconstrucciones_perfectas = diccionario_medidas["reconstrucciones_perfectas"]

    plt.subplot(2, 2, 1)
    plt.plot(list_epochs, list_training_times, c="red", marker='o', markersize=3, markerfacecolor="black")
    plt.xlabel("Número de epochs")
    plt.ylabel("Duración entrenamiento (s)")
    plt.title("Entrenamiento - Epochs")
    plt.xlim(left = 0)
    plt.ylim(bottom = 0)
    
    plt.subplot(2, 2, 2)
    plt.plot(list_epochs, list_media_precision, c="red", marker='o', markersize=3, markerfacecolor="black")
    plt.xlabel("Número de epochs")
    plt.ylabel("Media precisión descifrado")
    plt.title("Precisión media - Epochs")
    plt.xlim(left = 0)

    plt.subplot(2, 2, 3)
    plt.plot(list_epochs, list_media_distancias, c="red", marker='o', markersize=3, markerfacecolor="black")
    plt.xlabel("Número de epochs")
    plt.ylabel("Media distancias de Hamming")
    plt.title("Distancias de Hamming - Epochs")
    plt.xlim(left = 0)
    plt.ylim(bottom = 0)

    plt.subplot(2, 2, 4)
    plt.plot(list_epochs, list_reconstrucciones_perfectas, c="red", marker='o', markersize=3, markerfacecolor="black")
    plt.xlabel("Número de epochs")
    plt.ylabel("Reconstrucciones perfectas")
    plt.title("Reconstrucciones perfectas - Epochs")
    plt.xlim(left = 0)
    plt.ylim(0, n_mensajes)

    plt.subplots_adjust(left = 0.125, bottom = 0.11, right = 1.1, top = 0.9, wspace = 0.44, hspace = 0.45)
    plt.savefig(nombre_figura + ".png", bbox_inches='tight', dpi = 300)
    plt.show()

## Código principal

Aquí se define el código principal. Este es un main que tiene un bucle que ejecuta el código una vez. Con ayuda de bucles se puede conseguir que se ejecute más veces, y es lo que se hizo para obtener los resultados más cómodamente, pero en este caso, en pro de visualizar la gráfica resultante, se ha dejado una sola  ejecución.

Aquí es donde se definen los hiperparámetros que se utilizarán, como el tamaño del _batch_.

La ejecución tarda en completarse. Por ello, se han dejado los mensajes que Tensorflow y Keras imprimen por pantalla y se ha añadido uno, que sale en cada ronda de entrenamiento y da información acerca de por dónde y cómo va.

Además, aquí se recogen las métricas que se usan para dibujar las gráficas, los métodos de entrenamiento y evaluación devuelven el tiempo que duró el entrenamiento y las métricas recogidas durante la evaluación.

In [ ]:
if __name__ == '__main__':
    
    nombre_figura = "Figura 13 - Entrenamiento separado"
    res_file_name = "Resultados_13.txt"
    with open(res_file_name, "w") as f:
        f.write("-- RESULTADOS DEL EXPERIMENTO 1.3. --\n\n")

    n_mensajes = 10000
    bits = 32
    batch_size = 64
    adam_optimizer = 0.001
    muestras = 10

    epochs = 500
    step = 500
    total_epochs = 4000

    diccionario_medidas = {
        "epochs": [],
        "training_times": [],
        "media_precision": [],
        "media_distancias": [],
        "reconstrucciones_perfectas": []
    }

    with open(res_file_name, "a") as f:
        f.write(f"Número de mensajes = {n_mensajes}\n")
        f.write(f"Número de bits = {bits}\n")
        f.write(f"Tamaño del batch = {batch_size}\n")
        f.write(f"Adam optimizer learning rate = {adam_optimizer}\n")
        f.write(f"Epochs totales = {total_epochs}\n")
        f.write(f"Epochs iniciales = {epochs}\n")
        f.write("\n")
    
    while epochs <= total_epochs:
        training_time = entrenar(n_mensajes, bits, epochs, batch_size, adam_optimizer)
        res_list = evaluar(bits, muestras, res_file_name, epochs)
        
        media_precision = res_list[0]
        media_distancias = res_list[1]
        reconstrucciones_perfectas = res_list[2]

        diccionario_medidas["epochs"].append(epochs)
        diccionario_medidas["training_times"].append(training_time)
        diccionario_medidas["media_precision"].append(media_precision)
        diccionario_medidas["media_distancias"].append(media_distancias)
        diccionario_medidas["reconstrucciones_perfectas"].append(reconstrucciones_perfectas)

        epochs += step

    draw_graph(diccionario_medidas, n_mensajes, nombre_figura)

El resultado de la ejecución aparecerá encima de este texto y producirá una imagen al finalizar la ejecución con todos los resultados. Además, la imagen generada con las gráficas también queda guardada en el directorio de esta Jupyter Notebook.

En este escenario, los hiperparámetros no importan demasiado porque, al entrenar de manera separada, Alice y Bob no consiguen aprender a cifrar y descifdrar para el otro y por ello la precisión va a ser muy baja.